# Lab01

Unicode input: type "\\varphi" to obtain φ, "\\rho" for ρ, and similarly for other mathematical characters.

# Imports and utility functions

In [1]:
{-# LANGUAGE UnicodeSyntax, TypeSynonymInstances, FlexibleInstances #-}

import Data.List
import Control.Monad
import Test.QuickCheck
import System.IO.Unsafe

-- updating a function
update :: Eq a => (a -> b) -> a -> b -> a -> b
update ρ x v = \y -> if x == y then v else ρ y

-- useful for debugging
debug :: Show a => String -> a -> a
debug str x = seq (unsafePerformIO $ do putStr "<"; putStr str; putStr ": "; print x; putStr ">") x

todo :: a
todo = undefined

# Syntax

We define an inductive type for formulas of propositional logic.

In [1]:
-- propositional variable names are just strings
type PropName = String

data Formula =
      T
    | F
    | Prop PropName -- atomic formulas
    | Not Formula
    | And Formula Formula
    | Or Formula Formula
    | Implies Formula Formula
    | Iff Formula Formula
    deriving (Eq, Show)

We introduce a binary operator syntax:

In [1]:
infixr 8 /\, ∧
(/\) = And
(∧) = And -- input with "\and"

infixr 5 \/, ∨, ==>
(\/) = Or
(∨) = Or -- input with "\or"
(==>) = Implies

infixr 4 <==>, ⇔
(<==>) = Iff
(⇔) = Iff -- input with "\lr"

Example formulas:

In [1]:
p, q, r, s, t :: Formula

p = Prop "p"
q = Prop "q"
r = Prop "r"
s = Prop "s"
t = Prop "t"

satisfiable_formulas = [
    p ∧ q ∧ s ∧ p,
    T,
    p,
    Not p,
    (p ∨ q ∧ r) ∧ (Not p ∨ Not r),
    (p ∨ q) ∧ (Not p ∨ Not q)
  ]

unsatisfiable_formulas = [
    p ∧ q ∧ s ∧ Not p,
    T ∧ F,
    F,
    (p ∨ q ∧ r) ∧ Not p ∧ Not r,
    (p ⇔ q) ∧ (q ⇔ r) ∧ (r ⇔ s) ∧ (s ⇔ Not p)
  ]

## Random generation of propositional formulas (for testing)

In [1]:
instance Arbitrary Formula where
    arbitrary = sized f where
      
      f 0 = oneof $ map return $ [p, q, r, s, t] ++ [T, F]

      f size = frequency [
        (1, liftM Not (f (size - 1))),
        (4, do
              size' <- choose (0, size - 1)
              conn <- oneof $ map return [And, Or, Implies, Iff]
              left <- f size'
              right <- f $ size - size' - 1
              return $ conn left right)
        ]

Line 7: Use fmap
Found:
liftM
Why not:
fmap

# Semantics (EXERCISE)

Define the semantic function for propositional logic. *Hint*: proceed by structural induction on formulas, make sure the Iff case is not exponential!

In [1]:
-- truth valuations
type Valuation = PropName -> Bool

-- the evaluation function
eval :: Formula -> Valuation -> Bool
eval T _ = True
eval F _ = False
eval (Prop p) ρ = ρ p
eval (Not φ) ρ = not (eval φ ρ)
eval (And φ ψ) ρ = (eval φ ρ) && (eval ψ ρ)
eval (Or φ ψ) ρ = (eval φ ρ) || (eval ψ ρ)
eval (Implies φ ψ) ρ = eval (Or (Not φ) ψ) ρ
eval (Iff φ ψ) ρ = eval φ ρ == eval ψ ρ
eval _  _ = error "not a propositional formula"

Tests for the evaluation function:

In [1]:
ρ0 = const True
ρ1 = const False
ρ2 = update ρ0 "p" False

test_eval =
  eval (p ∧ Not p) ρ0 == False &&
  eval (p ∧ Not p) ρ1 == False &&
  eval (p ∨ q) ρ2 == True

quickCheck test_eval

-- check that the eval function is efficient
-- ifformula 3 == Iff (Iff (Iff T T) T) T
ifformula :: Int -> Formula
ifformula 0 = T
ifformula n = Iff (ifformula (n-1)) T

-- this should evaluate within a fraction of second
test_eval2 = eval (ifformula 23) (const True) == True

quickCheck test_eval2

+++ OK, passed 1 test.

+++ OK, passed 1 test.

## Satisfiable formulas

List of variables appearing in a formula:

In [1]:
variables :: Formula -> [PropName]
variables = nub . go where
  go T = []
  go F = []
  go (Prop p) = [p]
  go (Not φ) = go φ
  go (And φ ψ) = go φ ++ go ψ
  go (Or φ ψ) = go φ ++ go ψ
  go (Implies φ ψ) = go φ ++ go ψ
  go (Iff φ ψ) = go φ ++ go ψ
  go _ = error "not a propositional formula"

A trivial SAT solver based on truth tables:

In [1]:
type SATSolver = Formula -> Bool

-- the list of all valuations on a given list of variables
valuations :: [PropName] -> [Valuation]
valuations [] = [undefined]
valuations (x : xs) = concat [[update ρ x True, update ρ x False] | ρ <- valuations xs]

satisfiable :: SATSolver
satisfiable φ = or [eval φ ρ | ρ <- valuations (variables φ)]

## Tautologies (EXERCISE)

Write a program that checks whether a given input formula is a tautology. *Hint*: Reduce to satisfiability.

In [1]:
tautology :: Formula -> Bool
tautology φ = not $ satisfiable (Not φ)

# Normal forms

## Negation normal form (EXERCISE)

A formula of propositional logic is in *negation normal form* (NNF) if the only connectives are true (T), false (F), conjunction (And), disjunction (Or), and negation (Not), and moreover negation is only applied to atomic formulas:

In [1]:
is_nnf :: Formula -> Bool
is_nnf T = True
is_nnf F = True
is_nnf (Prop _) = True
is_nnf (Not (Prop _)) = True
is_nnf (And phi psi) = is_nnf phi && is_nnf psi
is_nnf (Or phi psi) = is_nnf phi && is_nnf psi
is_nnf (Implies phi psi) = False
is_nnf (Iff phi psi) = False
is_nnf (Not _) = False
is_nnf _ = error "not a propositional formula"

quickCheck $
  is_nnf (Not p ∧ (q ∨ (r ∧ s)))  -- NNF example
  && (not $ is_nnf $ Not (p ∨ q)) -- NNF non-example

Line 2: Move brackets to avoid $
Found:
is_nnf (Not p ∧ (q ∨ (r ∧ s))) && (not $ is_nnf $ Not (p ∨ q))
Why not:
is_nnf (Not p ∧ (q ∨ (r ∧ s))) && not (is_nnf $ Not (p ∨ q))

+++ OK, passed 1 test.

Write a function that turns an arbitrary formula to a *logically equivalent* one in NNF. What is the complexity of the NNF translation? *Hint*: Proceed by structural induction on formulas:

1. express Implies in terms of Not and Or,
2. express Iff in terms of And, Or, and Not, and
3. push negation inside the formula with De Morgan's laws.

In [1]:
nnf12 :: Formula -> Formula
nnf12 (Not phi) = Not (nnf12 phi)
nnf12 (And phi psi) = And (nnf12 phi) (nnf12 psi)
nnf12 (Or phi psi) = Or (nnf12 phi) (nnf12 psi)
nnf12 (Implies phi psi) = Or (nnf12 (Not phi)) (nnf12 psi)
nnf12 (Iff phi psi) = Or (And (nnf12 phi) (nnf12 psi)) (And (Not (nnf12 phi)) (Not (nnf12 psi)))
nnf12 phi = phi

nnf3 :: Formula -> Formula
nnf3 (Not T) = F
nnf3 (Not F) = T
nnf3 (Not (Not phi)) = nnf3 phi
nnf3 (And phi psi) = And (nnf3 phi) (nnf3 psi)
nnf3 (Or phi psi) = Or (nnf3 phi) (nnf3 psi)
nnf3 (Not (And phi psi)) = Or (nnf3 (Not phi)) (nnf3 (Not psi))
nnf3 (Not (Or phi psi)) = And (nnf3 (Not phi)) (nnf3 (Not psi))
nnf3 phi = phi

nnf :: Formula -> Formula
nnf = nnf3 . nnf12

Tests:

In [1]:
prop_nnf :: Formula -> Bool
prop_nnf φ = let ψ = nnf φ in is_nnf ψ && (tautology $ φ ⇔ ψ)

quickCheck prop_nnf

+++ OK, passed 100 tests.

## Disjunctive normal form (EXERCISE)

A *literal* is either  a propositional variable, or the negation of a propositional variable:

In [1]:
data Literal = Pos PropName | Neg PropName deriving (Eq, Show, Ord)

literal2var :: Literal -> PropName
literal2var (Pos p) = p
literal2var (Neg p) = p

A *clause* is a conjunction of literals. A formula of propositional logic is in *disjunctive normal form* (DNF) if it is a disjunction of clauses. It is customary to represent DNF formulas as lists of lists of literals:

In [1]:
type DNFClause = [Literal]
type DNF = [DNFClause]

dnf2formula :: [[Literal]] -> Formula
dnf2formula [] = F
dnf2formula lss = foldr1 Or (map go lss) where
  go [] = T
  go ls = foldr1 And (map go2 ls)
  go2 (Pos p) = Prop p
  go2 (Neg p) = Not (Prop p)

Write a function that turns an arbitrary formula to a *logically equivalent* one in DNF. What is the complexity of the DNF translation? *Hint*: Convert the formula to NNF first, then proceed by structural induction on NNF formulas (analogy with polynomial multiplication).

In [1]:
helper :: Formula -> DNF
helper T = [[]]
helper F = []
helper (Prop p) = [[Pos p]]
helper (Not (Prop p)) = [[Neg p]]
helper (And phi psi) = mergeDNF (helper phi) (helper psi)
helper (Or T psi) = [[]]
helper (Or phi T) = [[]]
helper (Or phi psi) = helper phi ++ helper psi

mergeDNF :: DNF -> DNF -> DNF
mergeDNF [] dnf2 = []
mergeDNF dnf1 [] = []
mergeDNF dnf1 dnf2 = [clause1 ++ clause2 | clause1 <- dnf1, clause2 <- dnf2]

dnf :: Formula -> DNF
dnf = helper . nnf

Tests:

In [1]:
test_dnf :: Formula -> Bool
test_dnf φ = tautology $ φ ⇔ (dnf2formula (dnf φ))

quickCheckWith (stdArgs {maxSize = 18}) test_dnf

+++ OK, passed 100 tests.

## Conjunctive normal form

The *conjunctive normal form* (CNF) is entirely dual to the DNF. In the next lab we will see how to convert a propositional formula to an *equisatisfiable* one in CNF, which is sufficient for SAT solving.

# DNF-based SAT solver (EXERCISE)

Write a SAT solver based on the following recipe:

1. Convert the input formula to DNF.
2. Find whether there exists a satisfiable clause.

*Hint*: A clause is satisfiable iff it does not contain the same literal both positively and negatively.

What is the complexity of this SAT solver compared to the one based on truth tables? Is it potentially more efficient on satisfiable or on unsatisfiable instances?

In [1]:
checkClause :: DNFClause -> Bool
checkClause clause =
    not $ any (\p -> Pos p `elem` clause && Neg p `elem` clause) lits
    where
        lits = map literal2var clause

checkClauses :: DNF -> Bool
checkClauses = or . (map checkClause)

sat_dnf :: SATSolver
sat_dnf =  checkClauses . dnf

Tests:

In [1]:
prop_sat_dnf :: Formula -> Bool
prop_sat_dnf phi = sat_dnf phi == satisfiable phi

quickCheckWith (stdArgs {maxSize = 20}) prop_sat_dnf

test_solver :: SATSolver -> Bool
test_solver solver = and $ map solver satisfiable_formulas ++ map (not . solver) unsatisfiable_formulas

quickCheck (test_solver sat_dnf)

+++ OK, passed 100 tests.

+++ OK, passed 1 test.

In [1]:
%%sh
stack install QuickCheck